In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from crawto.ml_flow import data_cleaning_flow
from tinydb import TinyDB

In [4]:
input_df = pd.read_csv("data/house-prices-advanced-regression-techniques/train.csv")
test= pd.read_csv("data/house-prices-advanced-regression-techniques/test.csv")

In [5]:
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data=input_df,
    problem="regression",
    target="SalePrice",
    features="infer",
    executor=executor,
    #db = "crawto.db"
)

[2020-04-28 00:13:06,806] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-28 00:13:06,811] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-28 00:13:07,099] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-28 00:13:07,152] INFO - prefect.TaskRunner | Task 'features': finished task run for task with final state: 'Success'
[2020-04-28 00:13:07,327] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-28 00:13:07,335] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-28 00:13:07,342] INFO - prefect.TaskRunner | Task 'input_data': finished task run for task with final state: 'Success'
[2020-04-28 00:13:07,349] INFO - prefect.TaskRunner | Task 'problem': finished task run for task with final state: 'Success'
[2020-04-28 00:13:07,368] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-28 00:13:07,377] INFO - prefect.TaskRunner | Task 'target': finished task run f

e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[2020-04-28 00:13:08,054] INFO - prefect.TaskRunner | Task 'fit_target_transformer': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,085] INFO - prefect.TaskRunner | Task 'transform_target': Starting task run...
[2020-04-28 00:13:08,088] INFO - prefect.TaskRunner | Task 'transform_target': Starting task run...
[2020-04-28 00:13:08,095] INFO - prefect.TaskRunner | Task 'transform_target': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,103] INFO - prefect.TaskRunner | Task 'transform_target': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,124] INFO - prefect.TaskRunner | Task 'save_data': Starting task run...
[2020-04-28 00:13:08,159] INFO - prefect.TaskRunner | Task 'save_data': Starting task run...
[2020-04-28 00:13:08,160] INFO - prefect.TaskRunner | Task 'fit_target_encoder': Starting task run...


e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
e:\projects\crawto\.venv\lib\site-packages\sklearn\preprocessing\_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[2020-04-28 00:13:08,250] INFO - prefect.TaskRunner | Task 'save_data': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,268] INFO - prefect.TaskRunner | Task 'save_data': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,904] INFO - prefect.TaskRunner | Task 'fit_target_encoder': finished task run for task with final state: 'Success'
[2020-04-28 00:13:08,928] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-28 00:13:08,940] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-28 00:13:09,071] INFO - prefect.TaskRunner | Task 'fit_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-28 00:13:09,115] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-28 00:13:09,121] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-28 00:

In [7]:
import sqlite3
sqlite3.connect("crawto.db").execute("Drop Table models")
from prefect.engine.executors import DaskExecutor
from crawto.meta_model import MetaModel, meta_model_flow
meta = MetaModel(problem="regression", db="crawto.db", use_default_models=True)
models = meta.models
executor = DaskExecutor()
meta_model_run = meta_model_flow.run(
    train_data = "transformed_train.df",
    valid_data = "transformed_valid.df",
    train_target = "train_target.df",
    valid_target="valid_target.df",
    db="crawto.db",
    executor=executor,
)

[2020-04-28 00:17:57,939] INFO - prefect.FlowRunner | Beginning Flow run for 'meta_model_flow'
[2020-04-28 00:17:57,943] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-28 00:17:58,065] INFO - prefect.TaskRunner | Task 'db': Starting task run...
[2020-04-28 00:17:58,080] INFO - prefect.TaskRunner | Task 'train_data': Starting task run...
[2020-04-28 00:17:58,084] INFO - prefect.TaskRunner | Task 'valid_target': Starting task run...
[2020-04-28 00:17:58,086] INFO - prefect.TaskRunner | Task 'db': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,098] INFO - prefect.TaskRunner | Task 'train_data': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,113] INFO - prefect.TaskRunner | Task 'train_target': Starting task run...
[2020-04-28 00:17:58,125] INFO - prefect.TaskRunner | Task 'valid_target': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,128] INFO - prefect.TaskRunner | Task 'valid_data': Starting 

e:\projects\crawto\.venv\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\projects\crawto\crawto\meta_model.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = model.fit(X=train_data, y=target)
e:\projects\crawto\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(


[2020-04-28 00:17:58,600] INFO - prefect.TaskRunner | Task 'fit_model[3]': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,653] INFO - prefect.TaskRunner | Task 'fit_model[4]': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,719] INFO - prefect.TaskRunner | Task 'fit_model[2]': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,785] INFO - prefect.TaskRunner | Task 'fit_model[5]': finished task run for task with final state: 'Success'
[2020-04-28 00:17:58,799] INFO - prefect.TaskRunner | Task 'fit_model': finished task run for task with final state: 'Mapped'
[2020-04-28 00:17:58,818] INFO - prefect.TaskRunner | Task 'predict_model': Starting task run...
[2020-04-28 00:17:58,871] INFO - prefect.TaskRunner | Task 'predict_model[0]': Starting task run...
[2020-04-28 00:17:58,906] INFO - prefect.TaskRunner | Task 'predict_model[3]': Starting task run...
[2020-04-28 00:17:58,913] INFO - prefect.TaskRunner | Tas

In [ ]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    